In [28]:

%load_ext autoreload
%autoreload 2

from test_bcpd_package import *
# import importlib
# importlib.reload(test_bcpd_package)

import numpy as np
import open3d as o3
import transformations as trans
from probreg import bcpd
from probreg import callbacks
import copy
import os
import csv

# My imports
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Visualization
to_plot = False

# Get list of processed frames in folder
output_path='.'
# target_fnames=[]
target_fnames = [fn for fn in glob.glob(os.path.join(output_path,'*.csv')) if ('Statistics' in fn)]

# Choose a source frame; the rest are targets
source_index = 3
source_fname = target_fnames.pop(source_index)
print("Source filename: ", source_fname)
print("Target fnames: ", target_fnames)

# Loop over the pairs
all_indices = None
for t_fname in target_fnames:
    # All the same source
    source, target = prepare_source_and_target_nonrigid_3d(source_fname,
                                                           t_fname,
                                                           0.005)
    if to_plot:
        cbs = [callbacks.Open3dVisualizerCallback(source, target)]
    else:
        cbs = []
    # Do BCPD
    print("Note: the value of w may need to be tweaked")
    tf_param = bcpd.registration_bcpd(source, target, w=1e-12,
                                      #gamma=10.0, #lmd=0.2, #k = 1e2,
                                      maxiter=100,
                                      callbacks=cbs)
    # Compute correspondence
    indices = correspondence_from_transform(tf_param, source, target)
    # save_indices(indices)
    if all_indices is None:
        all_indices = indices
    else:
        # print(all_indices.shape)
        # print(indices.shape)
        all_indices = np.hstack((all_indices, indices))

ind_fname = save_indices(all_indices)

Source filename:  .\Statistics for img3-emin.csv
Target fnames:  ['.\\Statistics for img0-emin.csv', '.\\Statistics for img1-emin.csv', '.\\Statistics for img2-emin.csv', '.\\Statistics for img4-emin.csv']
geometry::PointCloud with 89 points.
geometry::PointCloud with 81 points.
Note: the value of w may need to be tweaked
geometry::PointCloud with 89 points.
geometry::PointCloud with 113 points.
Note: the value of w may need to be tweaked
geometry::PointCloud with 89 points.
geometry::PointCloud with 96 points.
Note: the value of w may need to be tweaked
geometry::PointCloud with 89 points.
geometry::PointCloud with 1640 points.
Note: the value of w may need to be tweaked


In [30]:
## directly output in DLC format
all_fnames = [source_fname]
all_fnames.extend(target_fnames)
# print(all_fnames)
save_indices_DLC(all_indices, all_fnames)

[3, 0, 1, 2, 4]
5 images found ['.\\Statistics for img3-emin.csv', '.\\Statistics for img0-emin.csv', '.\\Statistics for img1-emin.csv', '.\\Statistics for img2-emin.csv', '.\\Statistics for img4-emin.csv']
Working on folder: .
Note: this does not have the exact DLC format, but is specific to Linux
[3, 0, 1, 2, 4]
['labeled-data/test_100frames.ome/img3.tif', 'labeled-data/test_100frames.ome/img0.tif', 'labeled-data/test_100frames.ome/img1.tif', 'labeled-data/test_100frames.ome/img2.tif', 'labeled-data/test_100frames.ome/img4.tif']


In [ ]:
all_indices.shape

In [7]:
indices.shape

(89, 1)

In [4]:
all_fnames

['.\\Statistics for img3-emin.csv',
 '.\\Statistics for img0-emin.csv',
 '.\\Statistics for img1-emin.csv',
 '.\\Statistics for img2-emin.csv',
 '.\\Statistics for img4-emin.csv']